In [49]:
import os

import pandas as pd
import plotly.express as px
# Install & Import Dependencies
import wandb

pd.options.plotting.backend = "plotly"
api = wandb.Api()

In [50]:
# Define WandB Parameters
entity = "phys_inversion"
project_prefix = "FINALLY__"
filters = {"state": {"$eq": "finished"}}

# Fetch all projects for the authenticated user
projects = api.projects(entity=entity)
projects = [project for project in projects if project_prefix in project.name]

rename_metrics = {
    "env/Env reach_success": "reach_success",
    "env/Env reach_distance": "reach_distance",
    "core/Episode Reward": "episode_reward"
}

In [51]:
# Fetch Data from WandB

def get_runs(entity, project, filters, keys=None):
    runs = api.runs(f"{entity}/{project}", filters=filters)
    data = []
    for run in runs:
        history = run.history(keys=keys, pandas=True)  # Adjust as needed

        # Fetch hyperparameters
        config = run.config
        history = pd.concat([history, pd.json_normalize(config, sep='.')], axis=1)

        history["run_name"] = run.name
        history["run_id"] = run.id

        data.append(history)
    return pd.concat(data).reset_index() if data else pd.DataFrame()


In [4]:
# ALL TOGETHER PER PROJECT
from tqdm import tqdm

os.makedirs(f"{project_prefix}/raw", exist_ok=True)
for project in tqdm(projects):
    df = get_runs(entity, project.name, filters)
    df.to_csv(f"{project_prefix}/raw/{project.name}.csv", index=False)

100%|██████████| 5/5 [05:48<00:00, 69.73s/it]


In [58]:
raw_csvs = os.listdir(f"{project_prefix}/raw")
grouped_dir = f"{project_prefix}/grouped"
os.makedirs(grouped_dir, exist_ok=True)
for project_file in raw_csvs:
    df = pd.read_csv(f"{project_prefix}/raw/{project_file}")
    df.rename(columns=rename_metrics, inplace=True)
    df[df['reach_success'] == "NaN"] = 0

    # multiply by 100 to get percentage
    df['reach_success'] = df['reach_success'] * 100

    # check if algo type is null, if so then if env.config.disable_discriminator is True then it's PPO else it's AMP
    df['algo_type'] = df.apply(lambda x:
                               x['algo_type'] if pd.notna(x['algo_type'])
                               else ('PPO' if x['env.config.disable_discriminator'] else 'AMP'), axis=1)
    df['prior'] = df['prior'].fillna(False)

    df = df[df.any(axis=1)]

    df_grouped = df.groupby(["algo_type", "prior", "use_perturbations"], dropna=False).agg(
        {"reach_success": ["mean", "std"]})
    df_grouped.columns = ["_".join(col).strip() for col in df_grouped.columns.to_flat_index()]
    df_grouped.reset_index(inplace=True)

    os.makedirs(project_prefix, exist_ok=True)
    df_grouped.to_csv(f"{grouped_dir}/{project_file}", float_format="%.4f")

In [156]:
import os
import pandas as pd
from glob import glob

# Folder containing the CSV files
folder_path = project_prefix  # Change this to the actual folder

# Read all CSV files
csv_files = glob(os.path.join(grouped_dir, "*.csv"))

keep_cols = ['algo_type', 'prior', 'use_perturbations', 'reach_success_mean', 'reach_success_std']
renamed_keep_cols = ['algo_type', 'prior', 'use_perturbations', 'mean', 'std']

# Dictionary to store data
data = {}

for file in csv_files:
    env_name = os.path.splitext(os.path.basename(file))[0].replace(project_prefix, '')  # Remove project prefix
    df = pd.read_csv(file)
    df = df.dropna(axis=1, how='all')  # Drop empty columns
    df = df[keep_cols]
    df.columns = renamed_keep_cols
    df['mean'] = pd.to_numeric(df['mean'], errors='coerce')  # Convert to numeric, coerce errors to NaN
    df['std'] = pd.to_numeric(df['std'], errors='coerce')  # Convert to numeric, coerce errors to NaN
    df['value'] = df.apply(
        lambda row: f"{row['mean']:.2f} ± {row['std']:.2f}" if pd.notna(row['std']) else f"{row['mean']:.2f}", axis=1)
    df['algo_str'] = df['algo_type'] + '_prior_' + df['prior'].map(str)
    df['env_perturb'] = env_name + '_perturb_' + df['use_perturbations'].map(str)

    for ep in df['env_perturb'].unique():
        data[ep] = df[df['env_perturb'] == ep].set_index(['algo_str', 'algo_type', 'prior'])['value']

# Combine data into a single DataFrame
combined_df = pd.concat(data, axis=1).reset_index()

In [160]:
def reorder_rows(df):
    # Define order of rows
    order = ['MaskedMimic_Inversion_prior_False',
             'MaskedMimic_Inversion_prior_True',
             'MaskedMimic_Prior_Only_prior_True',
             'MaskedMimic_Finetune_prior_False',
             'MaskedMimic_Finetune_prior_True',
             'PULSE_prior_False',
             'AMP_prior_False',
             'PureRL_prior_False',
             'PPO_prior_False', ]

    # Set order of rows
    df = df.sort_values('algo_str', key=lambda x: x.map(order.index))

    return df


def merge_rows(combined_df):
    # Merge 'PureRL_prior_False' and 'PPO_prior_False' rows
    df = combined_df.copy()
    if 'PureRL' in df['algo_type'].values and 'PPO' in df['algo_type'].values:
        pure_rl_row = df[df['algo_type'] == 'PureRL'].iloc[0]
        ppo_row = df[df['algo_type'] == 'PPO'].iloc[0]
        merged_row = ppo_row.combine_first(pure_rl_row)
        df = df[~df['algo_type'].isin(['PureRL', 'PPO'])]
        df = pd.concat([df, merged_row.to_frame().T], ignore_index=True)
    return df


def post_process_df(df):
    df = reorder_rows(df)
    df = merge_rows(df)
    return df

In [161]:
final_df = post_process_df(combined_df)
final_df

,algo_str,algo_type,prior,reach_perturb_False,reach_perturb_True,long_jump_perturb_False,long_jump_perturb_True,strike_perturb_False,strike_perturb_True,direction_facing_perturb_False,direction_facing_perturb_True,steering_perturb_False,steering_perturb_True
0,MaskedMimic_Inversion_prior_False,MaskedMimic_Inversion,False,95.37 ± 1.80,61.67 ± 38.73,99.81 ± 0.42,45.02 ± 48.36,70.27 ± 4.56,45.19 ± 11.00,79.17 ± 4.11,43.34 ± 34.04,97.05 ± 2.98,59.82 ± 43.96
1,MaskedMimic_Inversion_prior_True,MaskedMimic_Inversion,True,94.88 ± 1.99,65.06 ± 37.77,NaN,NaN,NaN,NaN,85.96 ± 3.67,55.98 ± 35.67,99.47 ± 0.46,52.02 ± 37.87
2,MaskedMimic_Prior_Only_prior_True,MaskedMimic_Prior_Only,True,24.77,23.99 ± 2.85,NaN,NaN,NaN,NaN,3.77,3.53 ± 1.55,0.48,0.22 ± 0.37
3,MaskedMimic_Finetune_prior_False,MaskedMimic_Finetune,False,93.70 ± 4.59,52.40 ± 33.50,47.38 ± 54.74,22.39 ± 34.85,79.61 ± 7.01,28.35 ± 23.19,90.67 ± 7.19,30.96 ± 32.58,99.23 ± 0.89,50.12 ± 43.89
4,MaskedMimic_Finetune_prior_True,MaskedMimic_Finetune,True,92.88 ± 3.42,66.31 ± 18.70,NaN,NaN,NaN,NaN,94.01 ± 7.44,30.14 ± 35.07,98.59 ± 1.66,45.91 ± 37.75
5,AMP_prior_False,AMP,False,57.14 ± 4.80,38.23 ± 16.67,76.26 ± 43.36,7.39 ± 21.94,30.92 ± 38.38,26.03 ± 31.65,4.30 ± 1.24,2.81 ± 2.36,5.83 ± 2.17,3.27 ± 2.34
6,PPO_prior_False,PPO,False,89.90 ± 3.25,66.83 ± 19.07,60.59 ± 53.98,11.81 ± 30.56,42.36 ± 35.51,41.11 ± 23.68,30.77 ± 38.23,6.61 ± 19.08,94.69 ± 10.67,12.91 ± 22.32


# Generate Tables

In [162]:
latex_output_dir = f"{project_prefix}/latex_tables"
os.makedirs(latex_output_dir, exist_ok=True)

## Table 1

In [168]:
def generate_table_1(final_df):
    # different methods - PPO, AMP, PULSE, PRIOR_ONLY, INVERSION prior FALSE
    # 5 envs, no perturbations
    table_1_df = final_df.copy()
    algos = ['MaskedMimic_Inversion_prior_False',
             # 'MaskedMimic_Inversion_prior_True',
             'MaskedMimic_Prior_Only_prior_True',
             # 'MaskedMimic_Finetune_prior_False',
             # 'MaskedMimic_Finetune_prior_True',
             'PULSE_prior_False',
             'AMP_prior_False',
             'PureRL_prior_False',
             'PPO_prior_False', ]
    table_1_df = table_1_df[table_1_df['algo_str'].isin(algos)]
    non_perturb_cols = [col for col in table_1_df.columns if 'perturb_False' in col]
    table_1_df = table_1_df[['algo_str'] + non_perturb_cols]

    renamed_cols = [col.replace('_perturb_False', '') for col in non_perturb_cols]
    # Rename columns
    table_1_df.columns = ['Method'] + renamed_cols
    # Rename algo names
    table_1_df['Method'] = table_1_df['Method'].replace({
        'MaskedMimic_Inversion_prior_False': 'Task Tokens (ours)',
        'MaskedMimic_Prior_Only_prior_True': 'MaskedMimic',
        'PULSE_prior_False': 'PULSE',
        'AMP_prior_False': 'AMP',
        'PureRL_prior_False': 'PureRL',
        'PPO_prior_False': 'PPO',
    })
    table_1_df = table_1_df[['Method', 'reach','steering', 'direction_facing', 'long_jump', 'strike']]
    table_1_df.columns = ['Method', 'Reach', 'Steering', 'Direction Facing', 'Long Jump', 'Strike']
    table_1_df = table_1_df.set_index('Method')
    return table_1_df

In [169]:
table_1 = generate_table_1(final_df)
table_1.to_latex(f"{latex_output_dir}/table_1.tex")
table_1

,Reach,Steering,Direction Facing,Long Jump,Strike
Method,,,,,
Task Tokens (ours),95.37 ± 1.80,97.05 ± 2.98,79.17 ± 4.11,99.81 ± 0.42,70.27 ± 4.56
MaskedMimic,24.77,0.48,3.77,NaN,NaN
AMP,57.14 ± 4.80,5.83 ± 2.17,4.30 ± 1.24,76.26 ± 43.36,30.92 ± 38.38
PPO,89.90 ± 3.25,94.69 ± 10.67,30.77 ± 38.23,60.59 ± 53.98,42.36 ± 35.51


,reach,long_jump,strike,direction_facing,steering
Method,,,,,
Task Tokens (ours),95.37 ± 1.80,99.81 ± 0.42,70.27 ± 4.56,79.17 ± 4.11,97.05 ± 2.98
MaskedMimic,24.77,NaN,NaN,3.77,0.48
AMP,57.14 ± 4.80,76.26 ± 43.36,30.92 ± 38.38,4.30 ± 1.24,5.83 ± 2.17
PPO,89.90 ± 3.25,60.59 ± 53.98,42.36 ± 35.51,30.77 ± 38.23,94.69 ± 10.67


In [ ]:
latex_table = combined_df.to_latex()

# Save to file
with open("output.tex", "w") as f:
    f.write(latex_table)

print("LaTeX table saved to output.tex")